In [1]:
from google.colab import drive
import os, sys
import csv

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tfb/shops_seoul.csv', encoding='utf-8')
# df.head(10)
# df = df.head(10)
df

In [3]:
df = df[['상호명', '상권업종중분류명', '상권업종소분류명', '표준산업분류명', '행정동명', '위도', '경도']]
df

,상호명,상권업종중분류명,상권업종소분류명,표준산업분류명,행정동명,위도,경도
0,비지트,한식,한식/백반/한정식,한식 음식점업,방배4동,37.488375,126.991394
1,쓰리에프,한식,한식/백반/한정식,한식 음식점업,사당2동,37.487105,126.980952
2,채움,한식,한식/백반/한정식,한식 음식점업,종로1.2.3.4가동,37.572387,126.981794
3,다향,한식,한식/백반/한정식,한식 음식점업,서초3동,37.483436,127.009382
4,고향생막걸리,유흥주점,민속주점,기타 주점업,압구정동,37.528073,127.031013
...,...,...,...,...,...,...,...
100778,브런치&다이닝37.5,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,하계1동,37.639226,127.074769
100779,수르점빵,제과제빵떡케익,제과점,제과점업,불광2동,37.621661,126.929479
100780,멍지트,커피점/카페,애견카페,비알콜 음료점업,신림동,37.487015,126.930092
100781,베이,제과제빵떡케익,케익전문점,제과점업,영등포동,37.523770,126.905662


In [4]:
df.columns = ['name',  # 상호명
              'cate_1',  # 중분류명
              'cate_2',  # 소분류명
              'cate_3',  # 표준산업분류명
              'dong',  # 행정동명
              'lon',  # 위도
              'lat'  # 경도
              ]

In [ ]:
df

#### step2. 키워드 리스트 컬럼 추가 : 상호명 + 중분류 + 소분류 + 표준산업분류명 

In [5]:
df['cate_3'] = df['cate_3'].fillna('') # 결측처리
df['cate_mix'] = df['name'] +' '+ df['cate_1'] +' '+ df['cate_2'] +' '+ df['cate_3'] 
df['cate_mix'] = df['cate_mix'].str.replace("/", " ")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
df

#### step3. 감정상태 입력받아 대응하는 맛집리스트업
##### - 설문조사 결과를 바탕으로 감정상태에 따른 음식 취향 선정 
존재

1.   기분이 좋을때 : 달달한 > 매콤한 


> 1.1. 달달한 음식 : '초콜릿' or '케익' 
> ##### - '달콤' 제외 : '매콤달콤' 처럼 중복 키워드 존재, '달콤커피' 처럼 
> ##### - '달달' 제외 : '더달달', '달달하네' 체인점 있음

2.   기분이 나쁠때 : 매콤한(압도적) > 달달한 

> 2.2. 매콤한 음식 : '매운' or '얼큰' 




In [7]:
is_happy = 'y' # happy : y, unhappy : n 
happy_keyword1 = '초콜릿'; happy_keyword2 = '케익'; unhappy_keyword1 = '매운'; unhappy_keyword2 = '얼큰'

if is_happy == 'y': # 기분이 좋을때
  df = df[df['cate_mix'].str.contains(happy_keyword1  ) | df['cate_mix'].str.contains(happy_keyword2  )]
elif is_happy == 'n': # 기분이 나쁠때
  df = df[df['cate_mix'].str.contains(unhappy_keyword1) | df['cate_mix'].str.contains(unhappy_keyword2)]
else:
  print("감정 키워드 누락!!")
df

,name,cate_1,cate_2,cate_3,dong,lon,lat,cate_mix
17,명봉화과점,제과제빵떡케익,제과점,제과점업,자양4동,37.538089,127.068653,명봉화과점 제과제빵떡케익 제과점 제과점업
19,대덕떡사랑,제과제빵떡케익,떡전문,제과점업,갈현2동,37.612630,126.915325,대덕떡사랑 제과제빵떡케익 떡전문 제과점업
49,샹제르망,제과제빵떡케익,제과점,제과점업,역촌동,37.606376,126.918180,샹제르망 제과제빵떡케익 제과점 제과점업
99,종로명인단팥빵,제과제빵떡케익,제과점,제과점업,종로1.2.3.4가동,37.569931,126.989582,종로명인단팥빵 제과제빵떡케익 제과점 제과점업
105,토끼의떡방아,제과제빵떡케익,떡전문,제과점업,망우본동,37.600429,127.100696,토끼의떡방아 제과제빵떡케익 떡전문 제과점업
...,...,...,...,...,...,...,...,...
100753,윤빵떡,제과제빵떡케익,제과점,제과점업,화곡8동,37.532549,126.848159,윤빵떡 제과제빵떡케익 제과점 제과점업
100758,39도스콘,제과제빵떡케익,제과점,제과점업,청운효자동,37.578652,126.968576,39도스콘 제과제빵떡케익 제과점 제과점업
100767,우진제과,제과제빵떡케익,제과점,제과점업,상봉2동,37.593056,127.078160,우진제과 제과제빵떡케익 제과점 제과점업
100779,수르점빵,제과제빵떡케익,제과점,제과점업,불광2동,37.621661,126.929479,수르점빵 제과제빵떡케익 제과점 제과점업


In [ ]:
df.reset_index(level=None, drop=False, inplace=True, col_level=0, col_fill='')

In [ ]:
df.index

RangeIndex(start=0, stop=315, step=1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도


count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(df['cate_mix']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

In [ ]:
len(place_simi_cate)
place_simi_cate_sorted_ind

array([[  0, 133, 156, ..., 242, 153, 314],
       [  1, 157, 133, ..., 242, 153, 314],
       [  2, 157, 133, ..., 242, 153, 314],
       ...,
       [312, 157, 133, ..., 166, 290, 314],
       [313, 133, 156, ..., 242, 153, 314],
       [314, 163,  57, ..., 191, 190,   0]])

In [ ]:
# 공식 1~5의 중요성을 짬뽕시키는 공식
# * 0.003 등의 가중치를 줘서 조절합니다.`
place_simi_co = (
                 + place_simi_cate * 0.3 # 공식 1. 카테고리 유사도
                #  + place_simi_review * 1 # 공식 2. 리뷰 텍스트 유사도
                #  + np.repeat([df['naver_blog_review_qty'].values], len(df['naver_blog_review_qty']) , axis=0) * 0.001  # 공식 3. 블로그 리뷰가 얼마나 많이 올라왔는지
                #  + np.repeat([df['naver_star_point'].values], len(df['naver_star_point']) , axis=0) * 0.005            # 공식 4. 블로그 별점이 얼마나 높은지
                #  + np.repeat([df['naver_star_point_qty'].values], len(df['naver_star_point_qty']) , axis=0) * 0.001    # 공식 5. 블로그 별점 평가가 얼마나 많이 됐는지
                 )

In [ ]:
# 아래 place_simi_co_sorted_ind 는 그냥 바로 사용하면 됩니다.
place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 

In [ ]:
df_sort = df.sort_values(by='lon' ,ascending=True).head(1) # 임시  
df_sort
df_sort.iloc[0]['name'].replace('\'','')
input_store = df_sort.iloc[0]['name']
input_store

'신원휘생태탕'

In [ ]:

df[df['name'].str.contains('신원휘생태탕')]

,index,name,cate_1,cate_2,cate_3,dong,lon,lat,cate_mix
203,56189,신원휘생태탕,일식/수산물,매운탕전문,한식 음식점업,시흥3동,37.434253,126.903882,신원휘생태탕 일식 수산물 매운탕전문 한식 음식점업


In [ ]:
# 최종 구현 함수
def find_simi_place(df, sorted_ind, place_name, top_n=10):
    
    print(place_name)
    place_title = df[df['name'] == place_name]
    place_index = place_title.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    print("place_title: ", place_title)
    print("place_index: ", place_index)
    print("similar_indexes: ", similar_indexes)
    return df.iloc[similar_indexes]

In [ ]:
df_result = find_simi_place(df, place_simi_co_sorted_ind, input_store, 5) # 
df_result

신원휘생태탕
place_title:       index    name  cate_1 cate_2   cate_3  dong        lon         lat  \
203  56189  신원휘생태탕  일식/수산물  매운탕전문  한식 음식점업  시흥3동  37.434253  126.903882   

                        cate_mix  
203  신원휘생태탕 일식 수산물 매운탕전문 한식 음식점업  
place_index:  [203]
similar_indexes:  [203 157 132 155 154]


,index,name,cate_1,cate_2,cate_3,dong,lon,lat,cate_mix
203,56189,신원휘생태탕,일식/수산물,매운탕전문,한식 음식점업,시흥3동,37.434253,126.903882,신원휘생태탕 일식 수산물 매운탕전문 한식 음식점업
157,45230,계양동태탕,일식/수산물,매운탕전문,한식 음식점업,방배1동,37.487401,126.994573,계양동태탕 일식 수산물 매운탕전문 한식 음식점업
132,36897,동태사랑,일식/수산물,매운탕전문,한식 음식점업,망우본동,37.599246,127.101492,동태사랑 일식 수산물 매운탕전문 한식 음식점업
155,44684,우리바다생태찌개,일식/수산물,매운탕전문,한식 음식점업,왕십리도선동,37.565452,127.035871,우리바다생태찌개 일식 수산물 매운탕전문 한식 음식점업
154,44489,강릉생태송파본점,일식/수산물,매운탕전문,한식 음식점업,방이2동,37.512644,127.109646,강릉생태송파본점 일식 수산물 매운탕전문 한식 음식점업
